In [1]:
import pandas as pd
# import numpy as np

In [2]:
def normalized_value_counts(series):
   counts = series.value_counts(dropna=False)
   percentages = series.value_counts(normalize=True, dropna=False).mul(100).round(2).astype(str) + '%'
   # result = pd.concat([counts, percentages], axis=1, keys=['N', '%'])
   result = pd.DataFrame({
      'N': counts,
      '%': percentages
   })
   return result

In [3]:
# codebook = pd.read_excel('codebook.xlsx')
from var_names import *
baseline = pd.read_csv('tba_data.csv')

In [4]:
baseline

,enumerator_name,location,gender,age_range,qual_life1,qual_life8,qual_life9,qual_life12,qual_life13,qual_life14,...,rapport3,partner_feedback,security1,security2,recognition,justice,recognition_justice,dignity,emergency_irm,comments
0,sb,gaziantep,male,25-55,good,not_at_all,moderate_amount,completely,a_little,a_little,...,yes,تشكر الجهود فقط,no,NaN,very_much,very_much,very_much,very_much,نقلة المنزل,لا
1,sb,gaziantep,male,25-55,good,not_at_all,moderate_amount,moderately,mostly,a_little,...,yes,كانو متعاونين إلى حد كبير,no,NaN,very_much,a_little,a_little,somewhat,صرف على أجور المعيشة والتنقل وقت الزلزال,يفضل ان يتم إعطاء الدعم المناسب لكل حالة حسب و...
2,sb,gaziantep,male,25-55,neither,moderate_amount,very_much,not_at_all,mostly,not_at_all,...,yes,مسرور من المشاركين بالمشروع,no,NaN,totally,totally,very_much,very_much,اجار منزل في اسطنبول,لا
3,sb,gaziantep,male,25-55,neither,not_at_all,very_much,a_little,moderately,a_little,...,yes,لا كل الشكر,no,NaN,very_much,very_much,totally,very_much,اجار للمنزل,يفضل ان يسرع المشروع بسبب الحاجة للدعم المادي
4,sb,gaziantep,female,over_55,very_poor,not_at_all,a_little,a_little,a_little,moderately,...,yes,مشكورين,no,NaN,very_much,very_much,very_much,somewhat,ساعدت فيه ابنتي ودفعت اجار البيت والفواتير,اتمنى ان يتم اضافة المساعدات الطبية من حيث الت...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,bh,urfa,male,25-55,good,extreme_amount,a_little,moderately,a_little,completely,...,somewhat,NaN,no,NaN,somewhat,not_at_all,not_at_all,not_at_all,NaN,NaN
102,bh,urfa,male,25-55,poor,not_at_all,a_little,moderately,a_little,completely,...,yes,NaN,no,NaN,totally,totally,totally,totally,المستلزمات الأساسية خلال فترة الزلزال التي خرج...,اتمنى ان يكون المبلغ جيد بما يكفي احتياجي لإجر...
103,bh,urfa,male,over_55,neither,not_at_all,a_little,not_at_all,mostly,not_at_all,...,yes,NaN,yes,اتخوف من تسرب المعلومات الشخصية للمشاركين وذلك...,somewhat,a_little,totally,somewhat,NaN,انا كفيل للأيتام وأرجو مساعدتي بتأمين أجور تدر...
104,bh,urfa,female,25-55,neither,a_little,moderate_amount,a_little,a_little,moderately,...,yes,NaN,yes,لا أعلم,very_much,somewhat,very_much,very_much,NaN,NaN


In [5]:
# processing order
def to_string_list(f):
   with open(f, 'r') as file:
      elements = file.read().strip().split('\n')
   
   formatted_element = [str(element) for element in elements]
   
   return formatted_element

In [6]:
lst = to_string_list('order.txt')
lst

['under_18',
 '18-24',
 '25-55',
 'over_55',
 'agree',
 'disagree',
 'male',
 'female',
 'other',
 'mersin',
 'hatay',
 'gaziantep',
 'urfa',
 'very_poor',
 'poor',
 'neither',
 'good',
 'very_good',
 '888',
 '999',
 'very_dissatisfied',
 'dissatisfied',
 'neither',
 'satisfied',
 'very_satisfied',
 '888',
 '999',
 'not_at_all',
 'a_little',
 'moderate_amount',
 'very_much',
 'extreme_amount',
 '888',
 '999',
 'not_at_all',
 'a_little',
 'moderately',
 'mostly',
 'completely',
 '888',
 '999',
 'never',
 'seldom',
 'quite_often',
 'very_often',
 'always',
 '888',
 '999',
 'none',
 'very_few',
 'some',
 'many',
 'very_many',
 '888',
 '999',
 'none',
 'a_little',
 'moderate_amount',
 'quite_a_lot',
 'very_much',
 '888',
 '999',
 'strongly_disagree',
 'disagree',
 'neither',
 'agree',
 'strongly_agree',
 '888',
 '999',
 'not_at_all',
 'a_little',
 'somewhat',
 'very_much',
 'totally',
 '888',
 '999',
 'test',
 'real',
 'yes',
 'no',
 '888',
 '999',
 'yes',
 'no',
 'somewhat',
 '888',
 '999

In [14]:
def tabulation(df, variable, text_file):
   series = df[variable]
   order_map = to_string_list(text_file)
   
   counts_order = []
   for x in order_map[:]:
      if x in series.unique() and x not in counts_order:
         counts_order.append(x)
         order_map.remove(x) 
   counts = normalized_value_counts(series=series).reindex(counts_order)
   counts.to_clipboard()
   
   return counts

In [16]:
question = 'qual_life2'
tabulation(baseline, question, 'order.txt')
# output_file = 'single_variables.xlsx'

,N,%
qual_life2,,
neither,25,23.58%
very_dissatisfied,7,6.6%
dissatisfied,27,25.47%
satisfied,41,38.68%
very_satisfied,6,5.66%


In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
# with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
#    for var in question:
#       Check if the variable exists in the dataframe
#       if var in baseline.columns:
         # Apply your function to the variable, and get the results
         # result_df = normalized_value_counts(baseline[var])
         # Write each DataFrame to a specific sheet
         # result_df.to_excel(writer, sheet_name=var)
      # else:
      #    print(f"Variable {var} not found in DataFrame.")

In [ ]:
baseline['network_size'].value_counts()

In [ ]:
# baseline['relative_support2'].value_counts(normalize=True) * 100